Attempt to apply a classifier into the data.

In [1]:
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'assignment_2'))
	print(os.getcwd())
except:
	pass

/Users/nwoodle/Developer/CSE258_recommender_system/assignment_2


In [2]:
import numpy as np 
import pandas as pd 
import pprint

pp = pprint.PrettyPrinter(indent=4)

In [3]:
# Data Preprocessing
closed_data = pd.read_csv("closed_data.csv")
closed_data = closed_data.loc[closed_data["district"].isin([1,2,3,4,5,6,7,8,9])]
print(f"Dataset size is {len(closed_data)}.")

Dataset size is 125177.


In [4]:
# Select columns
'''
Preprocessing:
	- Keep only closed case
	- Keep case with district data

Features:
	- Service name: service_name, onehot
	- Total population: total_pop2017
	- White population: white_pop2017
	- African American population: afram_pop2017
	- Hispanic population population: hisp_pop2017
	- Case origion: case_origin, onehot
	- Description sentiment: feature sentiment analysis, homebrewed
        - neg, neu, com, pos
    - Average income: avg_agi
'''
column_lst = ["total_pop2017", "white_pop2017", "afram_pop2017", "hisp_pop2017",\
              "avg_agi", "neg", "neu", "com", "pos"]
onehot_lst = ["service_name", "case_origin"]
prep_data = closed_data[column_lst]

In [5]:
# Onehot Process
onehot_data = closed_data[onehot_lst]
categorize_data = pd.get_dummies(onehot_data, prefix=onehot_lst)
dataset = pd.concat([prep_data, categorize_data], axis=1)

,total_pop2017,white_pop2017,afram_pop2017,hisp_pop2017,avg_agi,neg,neu,com,pos,service_name_--Select Answer--,...,case_origin_Letter,case_origin_Mobile,case_origin_Phone,case_origin_Referral Notice,case_origin_Route Slip,case_origin_Self-Generated,case_origin_Station 38 Incident Reports,case_origin_Walk-In,case_origin_Water Utilties Work Order (WAO),case_origin_Web
0,84025,62630,2972,62511,43025.30,0.000,0.000,0.0000,0.00,0,...,0,1,0,0,0,0,0,0,0,0
1,84025,62630,2972,62511,43025.30,0.000,0.000,0.0000,0.00,0,...,0,1,0,0,0,0,0,0,0,0
2,84025,62630,2972,62511,43025.30,0.000,1.000,0.0000,0.00,0,...,0,0,0,0,0,0,0,0,0,1
3,84025,62630,2972,62511,43025.30,0.000,1.000,0.0000,0.00,0,...,0,1,0,0,0,0,0,0,0,0
4,84025,62630,2972,62511,43025.30,0.000,1.000,0.0000,0.00,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125172,31728,26860,156,30002,31187.79,0.000,1.000,0.0000,0.00,0,...,0,1,0,0,0,0,0,0,0,0
125173,31728,26860,156,30002,31187.79,0.000,0.000,0.0000,0.00,0,...,0,1,0,0,0,0,0,0,0,0
125174,31728,26860,156,30002,31187.79,0.086,0.844,-0.2732,0.07,0,...,0,0,0,0,0,0,0,0,0,1
125175,31728,26860,156,30002,31187.79,0.000,0.000,0.0000,0.00,0,...,0,1,0,0,0,0,0,0,0,0
